In [1]:
from pathlib import Path

import quackosm as qosm
from lonboard import viz, basemap
from pooch import retrieve
import duckdb

### Download PBF file

In [2]:
DOWNLOAD_URL = (
    "https://download.bbbike.org/osm/bbbike/Miami/Miami.osm.pbf"
)
PBF_PATH = retrieve(
    url=DOWNLOAD_URL,
    path=Path(".").resolve(),
    known_hash=None,
)
PBF_PATH

'/mnt/ssd/dev/srai-tutorial/tutorial/51328953c16afbfa395e1a1f9177703d-Miami.osm.pbf'

### Convert PBF to GeoParquet file

In [3]:
result_file = qosm.convert_pbf_to_parquet(pbf_path=PBF_PATH)
result_file

Finished operation in 0:00:00

PosixPath('files/51328953c16afbfa395e1a1f9177703d-Miami_nofilter_noclip_compact.parquet')

In [4]:
duckdb.install_extension("spatial")
duckdb.load_extension("spatial")
duckdb.read_parquet(str(result_file)).project(
    "* EXCLUDE (geometry), ST_GeomFromWKB(geometry) geometry"
)

┌─────────────────┬───────────────────────────────────────────────────────────────────┬────────────────────────────────┐
│   feature_id    │                               tags                                │            geometry            │
│     varchar     │                       map(varchar, varchar)                       │            geometry            │
├─────────────────┼───────────────────────────────────────────────────────────────────┼────────────────────────────────┤
│ node/6063706036 │ {addr:city=Hialeah, addr:housenumber=1283, addr:postcode=33010,…  │ POINT (-80.261283 25.8339405)  │
│ node/6063706038 │ {addr:city=Hialeah, addr:housenumber=1040, addr:postcode=33010,…  │ POINT (-80.2609815 25.8361362) │
│ node/6063706046 │ {addr:city=Hialeah, addr:housenumber=515, addr:postcode=33010, …  │ POINT (-80.2713788 25.8349049) │
│ node/6063706049 │ {addr:city=Hialeah, addr:housenumber=553, addr:postcode=33010, …  │ POINT (-80.2704492 25.8305514) │
│ node/6063706055 │ {addr:city=H

### Convert PBF to GeoDataFrame

In [5]:
result_gdf = qosm.convert_pbf_to_geodataframe(pbf_path=PBF_PATH)
result_gdf

Finished operation in 0:00:00

,tags,geometry
feature_id,,
node/6063706036,"{'addr:city': 'Hialeah', 'addr:housenumber': '...",POINT (-80.26128 25.83394)
node/6063706038,"{'addr:city': 'Hialeah', 'addr:housenumber': '...",POINT (-80.26098 25.83614)
node/6063706046,"{'addr:city': 'Hialeah', 'addr:housenumber': '...",POINT (-80.27138 25.83490)
node/6063706049,"{'addr:city': 'Hialeah', 'addr:housenumber': '...",POINT (-80.27045 25.83055)
node/6063706055,"{'addr:city': 'Hialeah', 'addr:housenumber': '...",POINT (-80.26611 25.83160)
...,...,...
way/638557010,"{'addr:city': 'Coconut Grove', 'addr:housenumb...","POLYGON ((-80.26109 25.72117, -80.26109 25.721..."
way/651243299,"{'addr:city': 'Miami', 'addr:housenumber': '17...","POLYGON ((-80.22713 25.85956, -80.22713 25.859..."
way/436125306,"{'addr:city': 'Miami', 'addr:housenumber': '38...","POLYGON ((-80.25916 25.77295, -80.25932 25.772..."


In [7]:
viz(
    result_gdf.geometry,
    # optional style
    scatterplot_kwargs=dict(get_fill_color="#5c53a5"),
    path_kwargs=dict(get_color="#eb7f86"),
    polygon_kwargs=dict(stroked=False, get_fill_color="#f3e79b"),
    map_kwargs=dict(basemap_style=basemap.CartoBasemap.VoyagerNoLabels),
)

Map(basemap_style=<CartoBasemap.VoyagerNoLabels: 'https://basemaps.cartocdn.com/gl/voyager-nolabels-gl-style/s…